In [1]:
from train_class import AL_train_cifar

In [2]:
import random

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

#############################################################################################
# LOAD CONFIG
#############################################################################################
from config import Load_config
config = Load_config(len(x_train),x_train.shape[1:],num_classes,active_learning = False)

# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
indices = list(range(config["NUM_TRAIN"]))
random.shuffle(indices)
labeled_set = indices[:config["ADDENDUM"]]
unlabeled_set = indices[config["ADDENDUM"]:]

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


In [3]:
lfes =  AL_train_cifar(config, labeled_set)

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


wandb: Currently logged in as: afospinat (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model, losses, metrics, regularizers

In [5]:
for data in lfes.train_gen :
    
    image , c_true= data

    # Train 
    with tf.GradientTape() as tape:
        # predict the class and the features
        pred_c_f = lfes.Classifier(image)
        # get the class prediction form the list
        c_pred = pred_c_f[0]

        # if split the training stop gradients of the features
        if lfes.train_individual:
            features=[]
            for feat in pred_c_f[1:]:
                features.append(tf.stop_gradient(feat))
        else:
            features = pred_c_f[1:]

        # Predict the loss
        l_pred = lfes.Loss_net(features)
        # Compute the loss from the classification
        c_loss = lfes.Classification_loss_fn(c_true,c_pred)
        # Compute the loss from the Lossnet 
        l_loss, l_true = lfes.Loss_lerning_Class.run(c_true,c_pred,l_pred)
        # Compute the loss weighted sum of losses
        t_loss = (lfes.config.w_c_loss*c_loss)+(lfes.config.w_l_loss*l_loss)

    grads = tape.gradient(c_loss, lfes.Classifier.trainable_weights)#+lfes.Loss_net.trainable_weights)
    lfes.optimizer_fn.apply_gradients( zip(grads, lfes.Classifier.trainable_weights))#+lfes.Loss_net.trainable_weights))
    
    
    lfes.metrics["Accuracy Classification"].update_state(c_true, c_pred)
    lfes.metrics["MAE Learning loss loss"].update_state(metrics.mae(l_true, l_pred))
    lfes.metrics["Classification loss"].update_state(c_loss)
    lfes.metrics["Learning loss loss"].update_state(c_loss)
    lfes.metrics["Total loss"].update_state(c_loss)
    
    break

NotImplementedError: tf.GradientTape.gradients() does not support graph control flow operations like tf.cond or tf.while at this time. Use tf.gradients() instead. If you need this feature, please file a feature request at https://github.com/tensorflow/tensorflow/issues/new

In [6]:
pred_c_f = lfes.Classifier(image)
# get the class prediction form the list
c_pred = pred_c_f[0]

# if split the training stop gradients of the features
if lfes.train_individual:
    features=[]
    for feat in pred_c_f[1:]:
        features.append(tf.stop_gradient(feat))
else:
    features = pred_c_f[1:]

# Predict the loss
l_pred = lfes.Loss_net(features)
# Compute the loss from the classification
c_loss = lfes.Classification_loss_fn(c_true,c_pred)
# Compute the loss from the Lossnet 
l_loss, l_true = lfes.Loss_lerning_Class.run(c_true,c_pred,l_pred)
# Compute the loss weighted sum of losses
t_loss = (lfes.config.w_c_loss*c_loss)+(lfes.config.w_l_loss*l_loss)

In [7]:
t_loss

<tf.Tensor 'add_8:0' shape=() dtype=float32>

In [1]:
a={'b':'hola','c':'como','d':'estas'}

{'b': 4, 'c': 4, 'd': 5}

In [ ]:
l_loss